# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

import os
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
import joblib

from azureml.train.automl import AutoMLConfig


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#ws = Workspace.from_config()
ws = Workspace.get(name="quick-starts-ws-132278")

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [3]:
#  Create compute cluster

cluster_name = "cpu-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-12-29T07:17:45.577000+00:00', 'errors': None, 'creationTime': '2020-12-29T07:17:39.567563+00:00', 'modifiedTime': '2020-12-29T07:17:55.643850+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
# Get dataset
key = "Titanic"
description_text = "Titanic survival dataset"
if key in ws.datasets.keys(): 
    print("Found registered dataset")
    ds = ws.datasets[key] 
else:
    print("Cannot find registered dataset")
    
df = ds.to_pandas_dataframe()
print(df.shape)
df.head()

Found registered dataset
(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [5]:
# Use the clean_data function to clean your data.
label = "Survived"
data, y = clean_data(ds)
data[label] = y  # Add label column into data
print(data.shape)
data.head()

(891, 8)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,0,22.0,1,0,7.2500,2,0
1,1,0,38.0,1,0,71.2833,0,1
2,3,0,26.0,0,0,7.9250,2,1
3,1,0,35.0,1,0,53.1000,2,1
4,3,0,35.0,0,0,8.0500,2,0


In [6]:
# Save the train data to a csv to be uploaded to the datastore
if not os.path.isdir('data'):
    os.mkdir('data')    
pd.DataFrame(data).to_csv("data/train_data.csv", index=False)
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='Titanic', overwrite=True, show_progress=True)

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_8479e315b73f45f5a160880f97cb407d

In [7]:
# Upload the training data as a tabular dataset for access during training on remote compute
# automl doesnt accept dataframe or csv 
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('Titanic/train_data.csv'))

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
# Set parameters for AutoMLConfig
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='classification',
    compute_target=compute_target,
    debug_log = 'automl_errors.log',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5,
    **automl_settings)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
assert(remote_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)
print(best_run.properties)

In [ ]:
#TODO: Save the best model
best_run.download_file('outputs/model.pkl', 'outputs/model.pkl')
#best_run.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [ ]:
# Evaluate on val set
key = "Titanic"
description_text = "Titanic survival dataset"
if key in ws.datasets.keys(): 
    print("Found registered dataset")
    ds = ws.datasets[key] 
else:
    print("Cannot find registered dataset")

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
model_file = open('outputs/model.pkl', 'rb')
model = pickle.load(model_file)
model_file.close()
print("Accuracy:", accuracy_score(y_test, model.predict(x_test)))
print("Classification report:")
print(classification_report(y_test, model.predict(x_test)))
print("Confusion matrix:")
print(confusion_matrix(y_test, model.predict(x_test)))

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = remote_run.register_model(description='automl', tags= {'area': 'titanic'})
print(remote_run.model_id)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service